In [1]:
import pandas as pd
from utils import Timer

with Timer(f"Load train full"):
    train_data = pd.read_parquet("../nyc_taxi_fare/python/featuretools_humanknowledge_nyc_taxi_55M.parquet")


Load train full took 8.255218320060521 sec


In [2]:
train_data

,fare_amount,passenger_count,"BEARING(dropoff_latlong, pickup_latlong)","CITYBLOCK_DISTANCE(dropoff_latlong, pickup_latlong)",DAY(pickup_datetime),"DISTANCE_TO_LOCATION(dropoff_latlong, point=(40.639722, -73.778889))","DISTANCE_TO_LOCATION(pickup_latlong, point=(40.639722, -73.778889))","DISTANCE_TO_LOCATION(dropoff_latlong, point=(40.6892, -74.0445))","DISTANCE_TO_LOCATION(pickup_latlong, point=(40.6892, -74.0445))","DISTANCE_TO_LOCATION(dropoff_latlong, point=(40.6925, -74.168611))",...,"IS_IN_GEOBOX(dropoff_latlong, point1=(40.62, -73.85), point2=(40.7, -73.75))","IS_IN_GEOBOX(pickup_latlong, point1=(40.62, -73.85), point2=(40.7, -73.75))","IS_IN_GEOBOX(dropoff_latlong, point1=(40.7, -73.97), point2=(40.77, -73.9))","IS_IN_GEOBOX(pickup_latlong, point1=(40.7, -73.97), point2=(40.77, -73.9))",IS_WEEKEND(pickup_datetime),IS_WORKING_HOURS(pickup_datetime),MONTH(pickup_datetime),PART_OF_DAY(pickup_datetime),WEEKDAY(pickup_datetime),YEAR(pickup_datetime)
id,,,,,,,,,,,,,,,,,,,,,
43310508,30.2,1,102.817229,11.682842,1,14469.414565,14467.452444,15033.071503,15027.918488,13095.916771,...,False,True,False,False,False,False,1,midnight,3,2009
862908,15.0,1,35.967488,4.439169,1,14479.509503,14483.926434,15043.610395,15047.395190,13106.491962,...,False,False,False,False,False,False,1,midnight,3,2009
13073257,4.2,1,-139.932943,0.275526,1,14475.123838,14474.864057,15039.253161,15039.035907,13102.272002,...,False,False,False,False,False,False,1,midnight,3,2009
647957,5.8,2,11.803491,0.938679,1,14474.272235,14475.541493,15038.533411,15039.746867,13101.666276,...,False,False,False,False,False,False,1,midnight,3,2009
12655086,14.6,1,-118.186079,4.305175,1,14479.541327,14476.686179,15042.402716,15040.492224,13104.428568,...,False,False,True,False,False,False,1,midnight,3,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40210315,24.5,2,-15.549025,5.876328,30,5826.786496,5819.449711,18529.648942,18527.618863,14447.844211,...,False,False,False,False,False,False,6,midnight,1,2015
13957545,6.0,2,38.279771,1.241293,30,5819.263457,5818.123007,18527.673866,18526.343084,14455.381916,...,False,False,False,False,False,False,6,midnight,1,2015
48940597,33.5,1,-45.337623,10.384043,30,5822.504470,5814.442200,18520.887580,18523.837503,14452.079455,...,False,False,False,False,False,False,6,midnight,1,2015


In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.features.generators import AutoMLPipelineFeatureGenerator

test_sample = train_data.sample(frac = 0.1)
train_sample = train_data.drop(test_sample.index)

feature_generator = AutoMLPipelineFeatureGenerator(
                         enable_numeric_features=True,
                         enable_categorical_features=True,
                         enable_datetime_features=False,
                         enable_text_special_features=False,
                         enable_text_ngram_features=False,
                         enable_raw_text_features=False,
                         enable_vision_features=False,)

hyperparameters = {
    #'NN_TORCH': {}, 
    'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge',], 
    'CAT': {}, 
    'XGB': {}, 
    #'FASTAI': {}, 
    'RF': [
        {
            'criterion': 'gini', 
            'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}
        }, {
            'criterion': 'entropy', 
            'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}
        }, {
            'criterion': 'squared_error', 
            'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression']}
        },
    ], 
    'XT': [
        {
            'criterion': 'gini', 
            'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}
        }, {
            'criterion': 'entropy', 
            'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}
        }, {
            'criterion': 'squared_error', 
            'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression']}
        },
    ], 
    'KNN': [
        {
            'weights': 'uniform', 
            'ag_args': {'name_suffix': 'Unif'}
        }, {
            'weights': 'distance', 
            'ag_args': {'name_suffix': 'Dist'}
        },
    ],
}


model = TabularPredictor(label="fare_amount")
predictor = model.fit(train_sample, hyperparameters = hyperparameters)

No path specified. Models will be saved in: "AutogluonModels/ag-20221202_202806/"
Cannot interpret 'BooleanDtype' as a data type
Cannot interpret 'BooleanDtype' as a data type
Cannot interpret 'BooleanDtype' as a data type
Cannot interpret 'BooleanDtype' as a data type
Cannot interpret 'BooleanDtype' as a data type
Cannot interpret 'BooleanDtype' as a data type
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221202_202806/"
AutoGluon Version:  0.6.0
Python Version:     3.8.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Nov 8 23:39:32 UTC 2018
Train Data Rows:    48884359
Train Data Columns: 26
Label Column: fare_amount
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because 

		('category', []) :  1 | ['PART_OF_DAY(pickup_datetime)']
		('float', [])    : 13 | ['BEARING(dropoff_latlong, pickup_latlong)', 'CITYBLOCK_DISTANCE(dropoff_latlong, pickup_latlong)', 'DISTANCE_TO_LOCATION(dropoff_latlong, point=(40.639722, -73.778889))', 'DISTANCE_TO_LOCATION(pickup_latlong, point=(40.639722, -73.778889))', 'DISTANCE_TO_LOCATION(dropoff_latlong, point=(40.6892, -74.0445))', ...]
		('int', [])      :  6 | ['passenger_count', 'DAY(pickup_datetime)', 'HOUR(pickup_datetime)', 'MONTH(pickup_datetime)', 'WEEKDAY(pickup_datetime)', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('category', [])  :  1 | ['PART_OF_DAY(pickup_datetime)']
		('float', [])     : 13 | ['BEARING(dropoff_latlong, pickup_latlong)', 'CITYBLOCK_DISTANCE(dropoff_latlong, pickup_latlong)', 'DISTANCE_TO_LOCATION(dropoff_latlong, point=(40.639722, -73.778889))', 'DISTANCE_TO_LOCATION(pickup_latlong, point=(40.639722, -73.778889))', 'DISTANCE_TO_LOCATION(dropoff_latlong, point=(40.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

pred = predictor.predict(test_sample)
y_val = test_sample["fare_mount"]
    
rmse = np.sqrt(mean_squared_error(y_val.to_numpy(), pred.to_numpy()))
print('LightGBM RMSE', rmse)